## Modulos

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from pprint import pprint
import os

---

### Importacao e limpeza

In [36]:
df_raw = pd.read_csv('base.csv', sep=';', encoding='latin-1')
df_raw.head()

,Classe de Manutenção - Código,Classe de Manutenção - Descrição,Data de Entrada,Data de Saída,Funcionário - Código,Funcionário - Descrição,Sistema Veicular - Código,Sistema Veicular - Código.1,Subsistema Veicular - Código,Subsistema Veicular - Descrição,\nHoras,\n%Sistema Veicular,\n%Funcionário,\n%Data de Saída,\n%Data de Entrada,\n%Classe de Manutenção,\n% Geral
0,29,Corret. Emergencial,01/04/2020,01/04/2020,22955,Marcelo Venceslau da Silva,1400,SISTEMA HIDRÁULICO,1402,Cilindro hidráulico,2.35,100.0,100.0,2.10,0.11,0.0,0.0
1,29,Corret. Emergencial,01/04/2020,01/04/2020,30413,Agnaldo Aparecido Monteiro,1300,SISTEMA ELÉTRICO,1312,Indicadores Diversos,4.74,100.0,100.0,4.23,0.23,0.0,0.0
2,29,Corret. Emergencial,01/04/2020,01/04/2020,41598,Ademir de Jesus Aparecido,1400,SISTEMA HIDRÁULICO,1410,Mangueiras,2.99,100.0,100.0,2.67,0.15,0.0,0.0
3,29,Corret. Emergencial,01/04/2020,01/04/2020,50082,Alessandro Miranda,1000,PNEUS E AROS,1003,Pneus traseiros,0.29,100.0,100.0,0.26,0.01,0.0,0.0
4,29,Corret. Emergencial,01/04/2020,01/04/2020,52596,Josenaldo Silva Sousa,1300,SISTEMA ELÉTRICO,1301,Alternador,3.35,100.0,100.0,2.99,0.16,0.0,0.0


In [37]:
preprocessing_steps = {}

In [55]:
def column_cleaning(df):
    df_raw = df.copy()
    df_raw.columns = df_raw.columns.str.lower().str.strip('\n')
    rename = {
        'funcionário - código': 'cod_funcionario',
        'classe de manutenção - código': 'cod_classe_manutencao',
        'sistema veicular - código': 'cod_sistema',
        'subsistema veicular - código': 'cod_subsistema',
        'data de entrada': 'data_entrada',
        'data de saída': 'data_saida',
        'horas': 'horas',
        '%sistema veicular': '%sistema_veicular',
        '%funcionário': '%funcionario',
        '%data de entrada': '%data-entrada',
        '%data de saída': '%data_saida',
        '%classe de manutenção': '%classe',
        '% geral': '%geral'

    }
    # df = df_raw.rename(rename, axis=1)[['cod_classe_manutencao', 'cod_sistema', 'cod_subsistema', 'horas', '%sistema_veicular', '%funcionario', '%data-entrada', '%data_saida', '%classe', '%geral']]
    df = df_raw.rename(rename, axis=1)[['cod_classe_manutencao', 'cod_sistema', 'cod_subsistema', 'horas']]
    df['data_entrada'] = pd.to_datetime(df_raw['data de entrada'])
    # df['data_saida'] = pd.to_datetime(df_raw['data de saída'])
    return df
preprocessing_steps[0] = column_cleaning
df = column_cleaning(df_raw)

---

#### Dados duplicados (ou estranhos mesmo)

In [39]:
manutencao_mesmo_dia = (pd.to_numeric(df['data_saida'] - df['data_entrada'])) == 0
df['manutencao_mesmo_dia'] = manutencao_mesmo_dia

In [40]:
df[(df['manutencao_mesmo_dia'] == True) & (df['horas'] >= 24)]

,cod_classe_manutencao,cod_sistema,cod_subsistema,horas,%sistema_veicular,%funcionario,%data-entrada,%data_saida,%classe,%geral,data_entrada,data_saida,manutencao_mesmo_dia
2590,29,1100,1107,219.98,100.0,100.0,4.08,4.18,0.19,0.15,2020-04-30,2020-04-30,True
2591,29,1100,1107,219.98,100.0,100.0,4.08,4.18,0.19,0.15,2020-04-30,2020-04-30,True
2592,29,1100,1107,219.98,100.0,100.0,4.08,4.18,0.19,0.15,2020-04-30,2020-04-30,True
2593,29,1100,1107,219.98,100.0,100.0,4.08,4.18,0.19,0.15,2020-04-30,2020-04-30,True
2594,29,1100,1107,219.98,100.0,100.0,4.08,4.18,0.19,0.15,2020-04-30,2020-04-30,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27641,29,1100,1107,219.90,100.0,100.0,14.17,14.17,0.19,0.15,2020-12-31,2020-12-31,True
27642,29,1100,1107,219.98,100.0,100.0,14.18,14.18,0.19,0.15,2020-12-31,2020-12-31,True
27643,29,1100,1107,219.98,100.0,100.0,14.18,14.18,0.19,0.15,2020-12-31,2020-12-31,True
27644,29,1100,1107,219.98,100.0,100.0,14.18,14.18,0.19,0.15,2020-12-31,2020-12-31,True


Manutencoes que entraram e sairam no mesmo dia, mas duraram mais de 24 horas?<br>
Eles inventaram uma maquina do tempo?

In [41]:
# remocao dessas linhas
def remove_weird_rows(df):
    manutencao_mesmo_dia = (pd.to_numeric(df['data_saida'] - df['data_entrada'])) == 0
    df['manutencao_mesmo_dia'] = manutencao_mesmo_dia
    df.drop(df[(df['manutencao_mesmo_dia'] == True) & (df['horas'] >= 24)].index, inplace=True)
    df.drop(df[df['horas'] == 0].index, inplace=True)
    return df
df = remove_weird_rows(df)
preprocessing_steps[1] = remove_weird_rows

##### Adicao colunas data

In [56]:
def add_day_month(df):
    df['dia_semana'] = df['data_entrada'].dt.dayofweek
    df['mes'] = df['data_entrada'].dt.month
    return df
preprocessing_steps[2] = add_day_month
df = add_day_month(df)

#### Funções para avaliação dos resultados

In [57]:
results = {}

In [58]:
def evaluate(model, x_test, y_test, name='', dicto = results):
    predictions = model.predict(x_test)
    mape = mean_absolute_percentage_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    accuracy = round(1 - mape, 3)*100
    print('Performance')
    print(f'R2: {r2}')
    print(f'Erro medio (MAE): {mae}')
    print(f'Erro medio %(MAPE): {mape}')
    print(f'Precisão = {accuracy}%')
    
    results[name] = accuracy
    return accuracy

---

### Modelo 1b: Random Forest Regressor
    Features adicionais

In [59]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

#### Separacao em set de treino e teste

In [61]:
# x1 = df.drop(['horas', 'data_entrada', 'data_saida'], axis=1)
x1 = df.drop(['horas', 'data_entrada'], axis=1)
y1 = df['horas']

x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.15)

#### Parametros e fit dos sets

In [62]:
regressor = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
regressor.fit(x1_train, y1_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

#### Avaliacao dos resultados

In [63]:
df

,cod_classe_manutencao,cod_sistema,cod_subsistema,horas,data_entrada,dia_semana,mes
0,29,1400,1402,2.35,2020-01-04,5,1
1,29,1300,1312,4.74,2020-01-04,5,1
2,29,1400,1410,2.99,2020-01-04,5,1
3,29,1000,1003,0.29,2020-01-04,5,1
4,29,1300,1301,3.35,2020-01-04,5,1
...,...,...,...,...,...,...,...
51058,38,1300,1316,1.38,2020-12-23,2,12
51059,38,1600,1605,93.68,2020-12-24,3,12
51060,38,1000,1003,0.51,2020-12-28,0,12
51061,38,1,3,4.58,2020-12-31,3,12


In [64]:
evaluate(regressor, x1_test, y1_test, 'Random Forest Regressor')

Performance
R2: 0.790379134795153
Erro medio (MAE): 1.6022733059072867
Erro medio %(MAPE): 35823001869522.625
Precisão = -3582300186952162.5%


-3582300186952162.5

In [65]:
y1_test[:15]

31124     0.08
20741     5.51
45095     0.17
5539      0.65
4154      2.57
13394     2.99
21009     0.60
12113     0.79
49699     2.99
18577     0.48
40477     0.11
41184     0.43
26987     3.70
27280    12.04
50832     2.00
Name: horas, dtype: float64

---

### Validacao
    Dados 2020-2021

In [43]:
real_df = pd.read_csv('Base20-21.csv')
for i, f in preprocessing_steps.items():
    print(i, f)
    real_df = f(real_df)

0 <function column_cleaning at 0x0000015C865A6DC0>
1 <function remove_weird_rows at 0x0000015C865A6280>
2 <function add_day_month at 0x0000015C86165C10>
